In [ ]:
pip install --upgrade keras

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 23.8 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 2.15.0
    Uninstalling keras-2.15.0:
      Successfully uninstalled keras-2.15.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.15.0 requires keras<2.16,>=2.15.0, but you have keras 3.3.3 which is incompatible.


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import random
import tensorflow as tf
from pathlib import Path
from keras import applications
from keras import layers
from keras import losses
from keras import ops
from keras import optimizers
from keras import metrics
from keras import Model
from keras.applications import resnet


target_shape = (200, 200)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd "/content/drive/My Drive/DL Project"

Mounted at /content/drive
/content/drive/.shortcut-targets-by-id/1Vhu7c9TtXf-INnjjgmUD94b4o35wSPWM/DL Project


In [ ]:
base_dir = 'code/data/SiameseFormat/Augmented'

In [ ]:
anchor_images_path = base_dir + "/" +  "Anchor"
positive_images_path = base_dir + "/" + "Positive"
negative_images_path = base_dir + "/" + "Negative"
print(anchor_images_path)

code/data/SiameseFormat/Augmented/Anchor


In [ ]:

def preprocess_image(filename):
    """
    Load the specified file as a JPEG image, preprocess it and
    resize it to the target shape.
    """

    image_string = tf.io.read_file(filename)
    image = tf.image.decode_jpeg(image_string, channels=3)
    image = tf.image.convert_image_dtype(image, tf.float32)
    image = tf.image.resize(image, target_shape)
    return image


def preprocess_triplets(anchor, positive, negative):
    """
    Given the filenames corresponding to the three images, load and
    preprocess them.
    """

    return (
        preprocess_image(anchor),
        preprocess_image(positive),
        preprocess_image(negative),
    )


Let's setup our data pipeline using a zipped list with an anchor, positive,
and negative image filename as the source. The output of the pipeline
contains the same triplet with every image loaded and preprocessed.

In [ ]:
# We need to make sure both the anchor and positive images are loaded in
# sorted order so we can match them together.
anchor_images = sorted(
    [str(anchor_images_path + "/" + f) for f in os.listdir(anchor_images_path)]
)
print(anchor_images)

positive_images = sorted(
    [str(positive_images_path + "/" + f) for f in os.listdir(positive_images_path)]
)

negative_images = sorted(
    [str(negative_images_path + "/" + f) for f in os.listdir(negative_images_path)]
)

print(len(anchor_images))
print(len(positive_images))
print(len(negative_images))
image_count = len(anchor_images)

anchor_dataset = tf.data.Dataset.from_tensor_slices(anchor_images)
positive_dataset = tf.data.Dataset.from_tensor_slices(positive_images)
negative_dataset = tf.data.Dataset.from_tensor_slices(negative_images)

# # To generate the list of negative images, let's randomize the list of
# # available images and concatenate them together.
# rng = np.random.RandomState(seed=42)
# rng.shuffle(anchor_images)
# rng.shuffle(positive_images)

# negative_images = anchor_images + positive_images
# np.random.RandomState(seed=32).shuffle(negative_images)

# negative_dataset = tf.data.Dataset.from_tensor_slices(negative_images)
# negative_dataset = negative_dataset.shuffle(buffer_size=4096)

dataset = tf.data.Dataset.zip((anchor_dataset, positive_dataset, negative_dataset))
dataset = dataset.shuffle(buffer_size=1024)
dataset = dataset.map(preprocess_triplets)

# Let's now split our dataset in train and validation.
train_dataset = dataset.take(round(image_count * 0.8))
val_dataset = dataset.skip(round(image_count * 0.8))

train_dataset = train_dataset.batch(32, drop_remainder=False)
train_dataset = train_dataset.prefetch(tf.data.AUTOTUNE)

val_dataset = val_dataset.batch(32, drop_remainder=False)
val_dataset = val_dataset.prefetch(tf.data.AUTOTUNE)


['code/data/SiameseFormat/Augmented/Anchor/0_Camille_Pissarro_63_rotate90_flip_scale125_flip_mixup_flip_mixup.jpg', 'code/data/SiameseFormat/Augmented/Anchor/0_Edouard_Manet_18_rotate180.jpg', 'code/data/SiameseFormat/Augmented/Anchor/10000_Edouard_Manet_12_flip_mixup.jpg', 'code/data/SiameseFormat/Augmented/Anchor/10001_Vasiliy_Kandinskiy_20_scale75_rotate270_rotate90_flip.jpg', 'code/data/SiameseFormat/Augmented/Anchor/10002_Henri_Matisse_76_rotate270_scale75_rotate270_mixup.jpg', 'code/data/SiameseFormat/Augmented/Anchor/10003_El_Greco_83_rotate180_flip.jpg', 'code/data/SiameseFormat/Augmented/Anchor/10004_Paul_Klee_131_rotate180.jpg', 'code/data/SiameseFormat/Augmented/Anchor/10005_Amedeo_Modigliani_97_rotate180_mixup.jpg', 'code/data/SiameseFormat/Augmented/Anchor/10006_Rene_Magritte_94_rotate270_flip.jpg', 'code/data/SiameseFormat/Augmented/Anchor/10007_Piet_Mondrian_34_flip_scale75_flip.jpg', 'code/data/SiameseFormat/Augmented/Anchor/10008_Henri_de_Toulouse-Lautrec_48_scale75_mi

In [ ]:
print(type(anchor_images))


<class 'list'>


Let's take a look at a few examples of triplets. Notice how the first two images
look alike while the third one is always different.

In [ ]:
base_cnn = resnet.ResNet50(
    weights="imagenet", input_shape=target_shape + (3,), include_top=False
)

flatten = layers.Flatten()(base_cnn.output)
dense1 = layers.Dense(512, activation="relu")(flatten)
dense1 = layers.BatchNormalization()(dense1)
dense2 = layers.Dense(256, activation="relu")(dense1)
dense2 = layers.BatchNormalization()(dense2)
output = layers.Dense(256)(dense2)

embedding = Model(base_cnn.input, output, name="Embedding")

trainable = False
for layer in base_cnn.layers:
    if layer.name == "conv5_block1_out":
        trainable = True
    layer.trainable = trainable

In [ ]:

class DistanceLayer(layers.Layer):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)

    def call(self, anchor, positive, negative):
        ap_distance = ops.sum(tf.square(anchor - positive), -1)
        an_distance = ops.sum(tf.square(anchor - negative), -1)
        return (ap_distance, an_distance)


anchor_input = layers.Input(name="anchor", shape=target_shape + (3,))
positive_input = layers.Input(name="positive", shape=target_shape + (3,))
negative_input = layers.Input(name="negative", shape=target_shape + (3,))

distances = DistanceLayer()(
    embedding(resnet.preprocess_input(anchor_input)),
    embedding(resnet.preprocess_input(positive_input)),
    embedding(resnet.preprocess_input(negative_input)),
)

siamese_network = Model(
    inputs=[anchor_input, positive_input, negative_input], outputs=distances
)

In [ ]:

class SiameseModel(Model):
    """The Siamese Network model with a custom training and testing loops.

    Computes the triplet loss using the three embeddings produced by the
    Siamese Network.

    The triplet loss is defined as:
       L(A, P, N) = max(‖f(A) - f(P)‖² - ‖f(A) - f(N)‖² + margin, 0)
    """

    def __init__(self, siamese_network, margin=0.5):
        super().__init__()
        self.siamese_network = siamese_network
        self.margin = margin
        self.loss_tracker = metrics.Mean(name="loss")

    def call(self, inputs):
        return self.siamese_network(inputs)

    def train_step(self, data):
        with tf.GradientTape() as tape:
            loss = self._compute_loss(data)

        gradients = tape.gradient(loss, self.siamese_network.trainable_weights)

        self.optimizer.apply_gradients(
            zip(gradients, self.siamese_network.trainable_weights)
        )

        self.loss_tracker.update_state(loss)
        return {"loss": self.loss_tracker.result()}

    def test_step(self, data):
        loss = self._compute_loss(data)

        self.loss_tracker.update_state(loss)
        return {"loss": self.loss_tracker.result()}

    def _compute_loss(self, data):
        ap_distance, an_distance = self.siamese_network(data)

        loss = ap_distance - an_distance
        loss = tf.maximum(loss + self.margin, 0.0)
        return loss

    @property
    def metrics(self):
        return [self.loss_tracker]


In [ ]:
siamese_model = SiameseModel(siamese_network)
siamese_model.compile(optimizer=optimizers.Adam(0.001), metrics  = ['accuracy'])
siamese_model.fit(train_dataset, epochs=10, validation_data=val_dataset)

In [ ]:
siamese_model.save("/content/drive/My Drive/DL Project/code/SavedModels/triplets-2.keras")

In [ ]:
sample = next(iter(train_dataset))
visualize(*sample)

anchor, positive, negative = sample
anchor_embedding, positive_embedding, negative_embedding = (
    embedding(resnet.preprocess_input(anchor)),
    embedding(resnet.preprocess_input(positive)),
    embedding(resnet.preprocess_input(negative)),
)

In [ ]:
cosine_similarity = metrics.CosineSimilarity()

positive_similarity = cosine_similarity(anchor_embedding, positive_embedding)
print("Positive similarity:", positive_similarity.numpy())

negative_similarity = cosine_similarity(anchor_embedding, negative_embedding)
print("Negative similarity", negative_similarity.numpy())


Positive similarity: 0.999094
Negative similarity 0.9993547
